# Named Entity Recognition (NER)
In the last assignment, you tried out using statistical models for NLP tasks.

In this assignment you'll be experimenting with deep learning based models for a sequence labelling task: Named Entity Recognition

## Dataset and Task overview

First we provide a helper function which you can use for reading the data given.

Note: Feel free to augment the helper functions given in this notebook as per your need. As long as the overall objective is being met this shouldn't be an issue.

In [12]:
def read_file(filename):
    with open(filename, "r") as file:
        text = file.readlines()
    return text

def process_text(text):
    X = []
    Y = []
    sentenceX = []
    sentenceY = []
    for line in text:
        split = line.split(" ")
        if len(split) > 1:
            sentenceX.append(split[0])
            sentenceY.append(split[1].replace("\n", ""))
        else:
            X.append(sentenceX)
            Y.append(sentenceY)
            sentenceX = []
            sentenceY = []
    return X, Y

text = read_file("data/train.txt")
X, Y = process_text(text)

Following is an example to visualize what is happening here:

In [14]:
for i in range(len(X[1])):
    print(X[1][i], Y[1][i])

Chancellor O
of B-PP
the B-NP
Exchequer I-NP
Nigel B-NP
Lawson I-NP
's B-NP
restated I-NP
commitment I-NP
to B-PP
a B-NP
firm I-NP
monetary I-NP
policy I-NP
has B-VP
helped I-VP
to I-VP
prevent I-VP
a B-NP
freefall I-NP
in B-PP
sterling B-NP
over B-PP
the B-NP
past I-NP
week I-NP
. O


Notice how every token in the sentence has been assigned a label. 

These labels here are being referred to as the named entity.

Note that we are following a BIO Tagging scheme here.

Basically, every token is either the beginning (B) of a chunk, the continuity of a
chunk (I) or outside the chunk (O). E.g. "Barack Obama went to Greece today" -> “Barack
B-PER Obama I-PER went O to O Greece B-LOC today O." Of course, there are other types
of tagging schemes also possible like simply BO tagging, where I- is not explicitly tagged,
and all contiguous tokens of the same type are combined to extract one entity. In such a
schedule, the tagging will be “Barack PER Obama PER went O to O Greece LOC today O."

You can read more about it [here](https://lingpipe-blog.com/2009/10/14/coding-chunkers-as-taggers-io-bio-bmewo-and-bmewo/).

## Modelling

Your task in this assignment is to train a Deep Learning based model using the train set (feel free to split it into train and dev sets) and test out your models performance on a held out dataset (test set). A few points to note are the following:

- You need to use deep learning for this assignment. The allowed models are
CNN, LSTM. **However, you are NOT allowed to use any pretrained
Language Models such as BERT, ELMO, GPT.** You must train all models from
scratch. You are allowed to use pre-trained word vectors from word2vec,
Glove or FastText. If you wish to use any other pre-trained information, you
should ask on Piazza. To avoid confusion, you will be evaluated on the perfomance 
on one of your models. It would be good if you can show a comparison between the 
various settings you have tried however implementing one model completely would be 
sufficient as well. 
- You may like to create additional features for each token, e.g. whether the
token is capitalized or not, whether it’s a number or not etc. You may also try
features from lower level syntactic processing like POS tagging or shallow
chunking. (This step is optional and meant for your learning).
- You are welcome to use probabilistic models like CRF on top of deep learning
models. Example, read up on BiLSTM-CRF models for the
task of sequence labeling.

**Note: Use of only PyTorch is allowed for the assignment. Don't use SpaCy**

## Evaluation

In [27]:
# Here we have assumed that the predictions and the true labels are contained in a 1D array as shown below. 
# If you have a 2D array containing predictions of each sentence in a different array then please first flatten the array so that predictions are contained sequentially.

predY_eg = ["B-NN", "O", "B-PP", "I-PP"]
trueY_eg = ["B-NN", "O", "B-PP", "B-PP"]

THe metrics we'll be using are Micro and Macro F1 scores.

You can make use of the following code for calculating the scores which we'll be using for evaluating the performance. 

In [29]:
def get_scores(predY, trueY):
    from sklearn.metrics import f1_score
    trueY_O = [i for i, x in enumerate(trueY_eg) if x == "O"]
    predY = [predY[i] for i in range(len(predY)) if i not in trueY_O]
    trueY = [trueY[i] for i in range(len(trueY)) if i not in trueY_O]

    print("Micro F1 score: ", f1_score(trueY, predY, average="micro"))
    print("Macro F1 score: ", f1_score(trueY, predY, average="macro"))
    print("Average F1 score: ", (f1_score(trueY, predY, average="micro") + f1_score(trueY, predY, average="macro")) / 2)

get_scores(predY_eg, trueY_eg)

Micro F1 score:  0.6666666666666666
Macro F1 score:  0.5555555555555555
Average F1 score:  0.611111111111111


In [1]:
import torch
import numpy as np
from tqdm import tqdm
def load_vectors():
    with open('data/glove.6B.300d.txt','r') as f:
        lines = f.readlines()
    words = set()
    word_vec = {}
    for word in tqdm(lines):
                wrd = word.split()
                words.add(wrd[0])
                word_vec[wrd[0]] = torch.tensor(np.array(wrd[1:], dtype = np.float64)) 
    return words,word_vec

glove_vocab, word_vec_map = load_vectors()

100%|██████████| 400000/400000 [00:27<00:00, 14707.98it/s]


In [124]:
def load_data(file ='data/train.txt' ):
    with open(file) as f:
            train = f.read()
    sents = [i.split('\n') for i in train.split('. O')]

    sent_wrds = []
    sent_lables = []
    print("Seperating tags from words")
    for sent in tqdm(sents):
        wrds = [];labels = []
        for wrd in sent:
            temp = wrd.split()
            if len(temp)==2:
                wrds.append(temp[0].lower())
                labels.append(temp[1])
        if wrds != []:
            sent_wrds.append(wrds); sent_lables.append(labels)

    label_set = set()
    for lab in sent_lables:
        label_set = label_set.union(set(lab))
    print(label_set,'\nLength of label set : ',len(label_set))

    vocab_set = set()
    for wrds in sent_wrds:
        vocab_set = vocab_set.union(set(wrds))
    print('Length of vocab set : ',len(vocab_set))

    return sent_wrds,vocab_set,sent_lables,label_set

sent_wrds, vocab_set, sent_lables, label_set = load_data()

Seperating tags from words


100%|██████████| 8753/8753 [00:00<00:00, 61738.82it/s]

{'B-INTJ', 'I-INTJ', 'B-UCP', 'B-LST', 'I-ADVP', 'B-PP', 'B-VP', 'I-NP', 'I-PRT', 'B-NP', 'I-VP', 'B-ADVP', 'I-PP', 'O', 'B-CONJP', 'B-ADJP', 'I-ADJP', 'I-SBAR', 'B-PRT', 'I-CONJP', 'I-UCP', 'B-SBAR'} 
Length of label set :  22


Length of vocab set :  17258


In [96]:
t =torch.Tensor([len(i) for i in sent_wrds])
tot = 0
for i in t:
    if i >32:
        tot+=1
print(tot)
print(len(t))

1642
8733


In [125]:
print(sent_wrds[1471])
print(sent_lables[1471])

['what', 'makes', 'people', 'blurt', 'out', 'their', 'credit-card', 'numbers', 'to', 'a', 'caller', 'they', "'ve", 'never', 'heard', 'of', '?', 'do', 'they', 'really', 'believe', 'that', 'the', 'number', 'is', 'just', 'for', 'verification', 'and', 'is', 'simply', 'a', 'formality', 'on', 'the', 'road', 'to', 'being', 'a', 'grand-prize', 'winner', '?', 'what', 'makes', 'a', 'person', 'buy', 'an', 'oil', 'well', 'from', 'some', 'stranger', 'knocking', 'on', 'the', 'screen', 'door', '?', 'or', 'an', 'interest', 'in', 'a', 'retirement', 'community', 'in', 'nevada', 'that', 'will', 'knock', 'your', 'socks', 'off', ',', 'once', 'it', 'is', 'built', '?', 'because', 'in', 'the', 'end', ',', 'these', 'people', 'always', 'wind', 'up', 'asking', 'themselves', 'the', 'same', 'question', ':', '``', 'how', 'could', 'i', 'be', 'so', 'stupid', '?', "''", 'there', 'are', ',', 'unfortunately', ',', 'plenty', 'of', 'answers', 'to', 'that', 'question', '--', 'and', 'scam', 'artists', 'know', 'all', 'of', '

In [ ]:
#Padding and bringing sentence length down.
SENT_LEN = 20
for i in sent_wrds:
    

In [103]:
list(vocab_set)[0]

'Asbury'

In [108]:
tens = []
for wrd in glove_vocab:
    tens.append(word_vec_map[wrd])

In [114]:
t = torch.stack(tens)

In [ ]:
with open('data/glove.6B.300d.txt', 'r') as f:
    for i, line in enumerate(f):
        pass
n_vec = i + 1
hidden_dim = len(line.split(' ')) - 1
vecs = np.zeros((n_vec, hidden_dim), dtype=np.float32)
with open('data/glove.6B.300d.txt', 'r') as f:
    for i, line in enumerate(f):
        vecs[i] = np.array([float(n) for n in line.split(' ')[1:]], dtype=np.float32)

average_vec = np.mean(vecs, axis=0)
print(average_vec)

In [126]:
unks  = 0
count = 0
for wrd in vocab_set:
    if wrd not in glove_vocab:
        unks+=1
    if count<50:
        print(wrd)
        

becker
shivering
advises
priced
parcel
denied
finance
paying
noriega
enormous
exploits
phone-company
guideline
suntan
experience
reply
divvying
cup
reopen
resolve
help
immunities
suggesting
management-pilots
indulge
plus
32.8
once-fashionable
xenophobic
woodruff
kennedy
outgrew
dialogue
reignited
blame
ragan
depress
1.38
nimitz
future
t.
laurel
persons
generators
noise
walls
ratifying
johan
glorioso
inflationary
215.42
access
if
futile
positive
leo
u.s.-backed
workmen
claiming
mart
18th-century
against
lightest
headed
cities-abc
onto
index
loosen
biggest
witha
pair
fiercely
ignored
co-managing
unfavorable
britain
independent
hansen
3.23
mines
kyodo
relish
conspired
palladium
transition
transplants
exporting
slogan
tvx
wcrs
balmy
accounting
capitalize
198.41
963
tires
girder
definite
commentators
images
an
99
details
8.3
panamanian
switched
keizaikai
network-access
screeching
hockney
self-proclaimed
purchased
deadlocked
des
guildford
fastest
might
heady
events
institutions
straightforwa

In [127]:
print(unks)
print(len(vocab_set))
print(unks/len(vocab_set))
#Way too many unknowns

1464
17258
0.08483022366438753


In [129]:
torch.LongTensor([1,2])

tensor([1, 2])